## Objective

Let's use the subsetted features from L1 Lasso regularization and use them for OLS, which is more interpretable. Hopefully I won't be in a high variance situation.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import string
from StringIO import StringIO
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_validation import train_test_split

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
# rcParams['figure.figsize'] = 20, 5

from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [5]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)


(508653, 40)

In [6]:
print "I'm removing {0:.2f}% of my rows.".format((1 - df_outliers_removed.shape[0] / df_orig.shape[0]) * 100)

I'm removing 1.50% of my rows.


## Choosing columns

In [12]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'TYPE', 'SubmittedPhoto', 'Property_Type', 'Source']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [13]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(508653, 30)

## Dummify

In [14]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'TYPE', u'Property_Type', u'Source', u'school', u'housing'], dtype='object')

In [15]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

Zoning is baseline 0 5
other is baseline 1 5
Twitter is baseline 2 5
8_6th_grade is baseline 3 5
rent is baseline 4 5


In [16]:
df_dummified.shape

(508653, 232)

## Removing columns as per L1 results

In [19]:
col_blacklist = ['Property_Type_Address',
 'Property_Type_Intersection',
 'Source_Constituent Call',
 'SubmittedPhoto',
 'TYPE_ADA',
 'TYPE_Alert Boston',
 'TYPE_Animal Noise Disturbances',
 'TYPE_Automotive Noise Disturbance',
 'TYPE_BWSC General Request',
 'TYPE_BWSC Pothole',
 'TYPE_Big Buildings Online Request',
 'TYPE_Billing Complaint',
 'TYPE_Bridge Maintenance',
 'TYPE_CE Collection',
 'TYPE_Cemetery Maintenance Request',
 'TYPE_City/State Snow Issues',
 'TYPE_Contractor Complaints',
 'TYPE_Corporate or Community Group Service Day Clean Up',
 'TYPE_Downed Wire',
 'TYPE_Dumpster & Loading Noise Disturbances',
 'TYPE_Fire Department Request',
 'TYPE_Fire Hydrant',
 'TYPE_Fire in Food Establishment',
 'TYPE_Follow-Up',
 'TYPE_Food Alert - Confirmed',
 'TYPE_Food Alert - Unconfirmed',
 'TYPE_General Traffic Engineering Request',
 'TYPE_Ground Maintenance',
 'TYPE_HP Sign Application New',
 'TYPE_HP Sign Application Renewal',
 'TYPE_Heat/Fuel Assistance',
 'TYPE_Idea Collection',
 'TYPE_Knockdown Replacement',
 'TYPE_Loud Parties/Music/People',
 'TYPE_Mechanical',
 'TYPE_Misc. Snow Complaint',
 'TYPE_Mosquitoes (West Nile)',
 'TYPE_Municipal Parking Lot Complaints',
 'TYPE_New Tree Warrantee Inspection',
 'TYPE_News Boxes',
 'TYPE_No Utilities - Food Establishment - Electricity',
 'TYPE_No Utilities - Food Establishment - Flood',
 'TYPE_No Utilities - Food Establishment - Sewer',
 'TYPE_No Utilities - Food Establishment - Water',
 'TYPE_No Utilities Residential - Electricity',
 'TYPE_No Utilities Residential - Gas',
 'TYPE_No Utilities Residential - Water',
 'TYPE_OCR Metrolist',
 'TYPE_Occupying W/Out A Valid CO/CI',
 'TYPE_One Boston Day',
 'TYPE_PWD Graffiti',
 'TYPE_Parking Meter Repairs',
 'TYPE_Parks General Request',
 'TYPE_Pavement Marking Inspection',
 'TYPE_Phone Bank Service Inquiry',
 'TYPE_Planting',
 'TYPE_Poor Ventilation',
 'TYPE_Private Parking Lot Complaints',
 'TYPE_Public Events Noise Disturbances',
 'TYPE_Rat Bite',
 'TYPE_Rental Unit Delivery Conditions',
 'TYPE_Request for Litter Basket Installation',
 'TYPE_Roadway Flooding',
 'TYPE_Rooftop & Mechanical Disturbances',
 'TYPE_Schedule a Bulk Item Pickup SS',
 'TYPE_Senior Shoveling',
 'TYPE_Sewage/Septic Back-Up',
 'TYPE_Sidewalk Cover / Manhole',
 'TYPE_Sidewalk Repair (Make Safe)',
 'TYPE_Sign Shop WO',
 'TYPE_Snow Removal',
 'TYPE_Snow/Ice Control',
 'TYPE_Student Overcrowding',
 'TYPE_Transfer Not Completed',
 'TYPE_Undefined Noise Disturbance',
 'TYPE_Unit Pricing Wrong/Missing',
 'TYPE_Unsanitary Conditions - Employees',
 'TYPE_Unsanitary Conditions - Establishment',
 'TYPE_Unsanitary Conditions - Food',
 'TYPE_Utility Casting Repair',
 'TYPE_Valet Parking Problems',
 'TYPE_Walk-In Service Inquiry',
 'TYPE_Watermain Break',
 'TYPE_Work Hours-Loud Noise Complaints',
 'TYPE_Yardwaste Asian Longhorned Beetle Affected Area',
 'bedroom',
 'bedroom_std_dev',
 'earned_income_per_capita',
 'housing_own',
 'housing_std_dev',
 'income',
 'income_std_dev',
 'is_description',
 'poverty_pop_below_poverty_level',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_ssi',
 'queue_wk',
 'race_asian',
 'race_black',
 'race_hispanic',
 'race_other',
 'race_white',
 'rent',
 'rent_std_dev',
 'school_0_none',
 'school_11_9th_grade',
 'school_13_11th_grade',
 'school_14_12th_grade_no_diploma',
 'school_15_hs_diploma',
 'school_18_some_college_no_degree',
 'school_19_associates',
 'school_20_bachelors',
 'school_21_masters',
 'school_22_professional_school',
 'school_std_dev',
 'value',
 'value_std_dev']

In [20]:
df_dummified_and_filtered = df_dummified.drop(col_blacklist, axis=1)
df_dummified_and_filtered.shape

(508653, 114)

## Running a model

In [21]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [22]:
df_dummified_and_filtered.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified_and_filtered.columns]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified_and_filtered.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified_and_filtered.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [24]:
col_list = ' + '.join(df_dummified_and_filtered.drop('COMPLETION_HOURS_LOG_10', axis=1))

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                              
===================================================================================
Dep. Variable:     COMPLETION_HOURS_LOG_10   R-squared:                       0.556
Model:                                 OLS   Adj. R-squared:                  0.556
Method:                      Least Squares   F-statistic:                     4502.
Date:                     Thu, 23 Feb 2017   Prob (F-statistic):               0.00
Time:                             01:28:32   Log-Likelihood:            -4.5750e+05
No. Observations:                   406922   AIC:                         9.152e+05
Df Residuals:                       406808   BIC:                         9.165e+05
Df Model:                              113                                         
Covariance Type:                 nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         1.9291      0.004    476.895      0.000         1.921     1.937
queuewkopen                                    6.495e-05   7.77e-07     83.539      0.000      6.34e-05  6.65e-05
TYPEAbandonedBicycle                              0.5255      0.025     20.759      0.000         0.476     0.575
TYPEAbandonedBuilding                             0.7542      0.038     19.788      0.000         0.679     0.829
TYPEAbandonedVehicles                             0.4962      0.010     50.029      0.000         0.477     0.516
TYPEAnimalFound                                  -0.6837      0.077     -8.933      0.000        -0.834    -0.534
TYPEAnimalGenericRequest                         -0.4907      0.027    -18.256      0.000        -0.543    -0.438
TYPEAnimalLost                                   -0.8443      0.089     -9.473      0.000        -1.019    -0.670
TYPEBedBugs                                       0.7551      0.027     27.528      0.000         0.701     0.809
TYPEBicycleIssues                                 1.1198      0.097     11.537      0.000         0.930     1.310
TYPEBreatheEasy                                   0.8614      0.028     31.098      0.000         0.807     0.916
TYPEBuildingInspectionRequest                     0.6404      0.013     49.212      0.000         0.615     0.666
TYPECallLog                                      -3.6007      0.019   -190.349      0.000        -3.638    -3.564
TYPECarbonMonoxide                               -1.3583      0.090    -15.021      0.000        -1.535    -1.181
TYPECatchbasin                                    0.2447      0.028      8.876      0.000         0.191     0.299
TYPECheckin                                      -4.0755      0.149    -27.346      0.000        -4.368    -3.783
TYPEChronicDampnessMold                           0.8166      0.031     26.601      0.000         0.756     0.877
TYPEConstructionDebris                           -1.4697      0.064    -23.050      0.000        -1.595    -1.345
TYPEContractorsComplaint                          0.6417      0.026     24.426      0.000         0.590     0.693
TYPECrossMeteringSubMetering                      0.3441      0.034     10.244      0.000         0.278     0.410
TYPEEgress                                        0.6415      0.058     11.137      0.000         0.529     0.754
TYPEElectrical                                    0.5807      0.021     27.725      0.000         0.540     0.622
TYPEEmptyLitterBasket                            -0.9384      0.013    -73.004      0.000        -0.964    -0.913
TYPEEquipmentRepair                               0.4307      0.

### Getting adjusted $R^2$ on test set

In [ ]:
# est.save('../data/model_completion_time.pkl')
# model = sm.load('../data/model_completion_time.pkl')

In [25]:
y_pred = est.predict(X_test)

In [26]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.5506924222357249

In [27]:
mean_squared_error(y_test, y_pred)**0.5

0.75015459728427791

## Interpreting model

Which features are most associated with completion time?

In [29]:
df = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df.columns][1:]
df['coef_abs'] = pd.np.abs(df.coef)
df = df.sort_values('P>|t|')

In [33]:
df.head(300)

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,1.9291,0.004,476.895,0.000,1.921 1.937,1.9291
81,TYPERoadwayRepair,1.3938,0.031,44.946,0.000,1.333 1.455,1.3938
80,TYPERequestsforTrafficSignalStudiesorReviews,0.3577,0.025,14.508,0.000,0.309 0.406,0.3577
79,TYPERequestsforStreetCleaning,-1.1457,0.007,-159.356,0.000,-1.160 -1.132,1.1457
78,TYPERequestforSnowPlowingEmergencyResponder,-0.5177,0.024,-22.026,0.000,-0.564 -0.472,0.5177
77,TYPERequestforSnowPlowing,-0.6040,0.010,-61.386,0.000,-0.623 -0.585,0.6040
76,TYPERequestforRecyclingCart,0.4124,0.008,50.714,0.000,0.396 0.428,0.4124
75,TYPERequestforPotholeRepair,-0.3709,0.008,-44.433,0.000,-0.387 -0.355,0.3709
74,TYPERecyclingCartReturn,-0.7938,0.019,-41.370,0.000,-0.831 -0.756,0.7938
73,TYPERecyclingCartInquiry,1.0564,0.029,36.984,0.000,1.000 1.112,1.0564


The features most associated with completion time are the categories, which makes intuitive sense, since there are many categories and they don't overlap with each other.

It is comforting that "Water in Gas High Priority" and "Short Measure Gas" are associated with faster completion time.

What about the non-category features?

In [34]:
df[~df.coef_name.str.contains('TYPE')]

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,1.929100,4.000000e-03,476.895,0.0,1.921 1.937,1.929100
112,SourceCitizensConnectApp,-0.063500,4.000000e-03,-16.305,0.0,-0.071 -0.056,0.063500
1,queuewkopen,0.000065,7.770000e-07,83.539,0.0,6.34e-05 6.65e-05,0.000065
113,SourceSelfService,0.040800,4.000000e-03,11.318,0.0,0.034 0.048,0.040800


In [197]:
df.sort_values('coef_abs', ascending=False)[~df.coef_name.str.contains('TYPE')]

/home/ec2-user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,1.927300,4.000000e-03,474.854,0.0,1.919 1.935,1.927300
115,neighborhoodfromzipNorthEnd,-0.065800,9.000000e-03,-7.152,0.0,-0.084 -0.048,0.065800
112,SourceCitizensConnectApp,-0.062400,4.000000e-03,-16.005,0.0,-0.070 -0.055,0.062400
114,neighborhoodfromzipEastBoston,0.057300,5.000000e-03,10.663,0.0,0.047 0.068,0.057300
113,SourceSelfService,0.041500,4.000000e-03,11.510,0.0,0.034 0.049,0.041500
1,queuewkopen,0.000065,7.770000e-07,83.586,0.0,6.34e-05 6.65e-05,0.000065


To interpet the above coefficients, holding the other variables constant:
- if someone lives in the North End, their issue is associated with a 1 hour _decrease_ in completion time
- if someone lives in the East Boston, their issue is associated with a 1 hour _increase_ in completion time
- if someone sends an issue from the mobile app, their issue is associated with a 1 hour _decrease_ in completion time
- if someone sends an issue from the mobile app, their issue is associated with a 1 hour _increase_ in completion time

## Future steps:
- Use dataset before June 2016, as there are more issues that haven't been completed since then
- Make a separate model for each category. Then some of my engineered features are more likely to be significant, for some of the categories.